In [303]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [223]:
pd.set_option('display.max_colwidth', 1000)

In [224]:
PATH_TO_DATA = './data'

In [225]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [6]:
files[0]

'./data/ng_0.jsonlines.zip'

In [226]:
data = pd.read_json(files[0], lines=True)

In [227]:
data.loc[0:10, ["keywords"]]

,keywords
0,"[яблоко, молодежь, молодежное яблоко]"
1,"[газпром, газ]"
2,"[франсуа рабле, сервантес, шекспир, конан дойл, михаил булгаков, александр грин, борхес, босх, маркес, герман гессе, голландская живопись, гаргантюа и пантагрюэль, дон кихот, мастер и маргарита, москва, россия, история, поэзия, шотландия, баллада, пере]"
3,"[владивосток, суд, ким, футина, выборы, боевое братство]"
4,"[новая москва, подмосковье, благоустройство, тинао, городская среда, парки]"
5,"[автоспорт, формула1, гонки, хэмилтон, квят, сша]"
6,"[ведущие политики, рейтинг, февраль]"
7,"[законы, культура, концепция, уголовные дела, платформа, аресты]"
8,"[сирия, вооруженный конфликт, газ, углеводороды, турецкий поток]"
9,"[фсб, иг, калужская область]"


In [243]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

In [241]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


Давайте посмотрим какие обычно ключевые слова. 

# Тупое решение.

Давайте не будем думать, а попробуем сразу придумать какое-то решение.

Возьмем первые 5 слов из заголовка.

In [244]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:5]))

Precision -  0.06
Recall -  0.06
F1 -  0.06
Jaccard -  0.03


Или 10.

In [245]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:10]))

Precision -  0.06
Recall -  0.07
F1 -  0.06
Jaccard -  0.03


Теперь попробуем взять самые частотные слова.

In [257]:
evaluate(data['keywords'], data['content'].apply(lambda x: [x[0] for x in Counter(x.lower().split()).most_common(10)]))

Precision -  0.02
Recall -  0.04
F1 -  0.02
Jaccard -  0.01


Или вообще рандомные слова.

In [278]:
evaluate(data['keywords'], data['content'].apply(lambda x: np.random.choice(list(set(x.lower().split())), 10)))

Precision -  0.0
Recall -  0.01
F1 -  0.01
Jaccard -  0.0


Теперь давайте посмотрим, что вообще извлекается.

In [272]:
data['title'].apply(lambda x: x.lower().split()[:10]).head(10)

0                         ["молодежное, "яблоко":, оппозиционная, деятельность, становится, опасной]
1                                                                 ["газпрома", на, всех, не, хватит]
2                                                   [бесконечная, партия, в, четырехмерные, шахматы]
3    [экс-депутат,, осужденная, за, фальсификацию, выборов,, оказалась, членом, "боевого, братства"]
4                               [новая, москва, останется, территорией, экологической, безопасности]
5                                [f1., гран-при, сша, прошел, без, четырех, машин, и, со, «стопкой»]
6                                          [100, ведущих, политиков, россии, в, феврале, 2018, года]
7                                               [закон, "о, культуре", принимают, на, фоне, арестов]
8                                    [насколько, реальна, газовая, подоплека, сирийского, конфликта]
9                                  [фсб:, в, калужской, области, задержаны, четверо, участн

In [275]:
data['content'].apply(lambda x: [x[0] for x in Counter(x.lower().split()).most_common(10)]).head(10)

0                                         [в, и, на, что, не, –, его, «молодежное, с, из]
1                                              [в, и, на, –, млрд., куб., по, к, газа, м]
2                                                   [в, –, и, не, я, но, что, это, на, с]
3                                         [в, на, и, ким, –, по, что, он, видео, зинаиды]
4                             [в, и, на, новой, площадью, –, с, развития, москвы, парков]
5                                               [в, на, и, не, с, но, уже, что, у, гонки]
6                                    [на, в, (с, место)., и, рф, позиции, влияние, по, с]
7                                          [в, и, –, по, с, культуре, будет, из, не, как]
8                                                [в, и, на, с, что, для, по, –, газа, не]
9    [в, террористической, организации, –, задержаны, рф, он, по, возглавляемой, «спящей]
Name: content, dtype: object

Когда выбираются самые частотные слова, выбираются всякие стоп-слова. Также из-за плохой токенизации некоторые слова в обоих списках - пунктуация или слова с пунктуацией на концах.

## Очищаем текст.

In [281]:
from string import punctuation
from nltk.corpus import stopwords
punct = set(punctuation+'«»—…“”*№–')
stops = set(stopwords.words('russian'))

def delete_punct_on_edges(word):
    if len(word) < 2:
        
        if not word or word in punct:
            return None
        else:
            # если какая-то буква то выдаем
            return word

    if word[0] not in punct:

        if word[-1] not in punct:
            return word
        else:
            return delete_punct_on_edges(word[:-1])
    else:
        return delete_punct_on_edges(word[1:])
      


def normalize(text):
    
    words = [delete_punct_on_edges(word) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [286]:
data['content_norm'] = data['content'].apply(normalize)

In [289]:
data['title_norm'] = data['title'].apply(normalize)

In [291]:
data['title_norm'].head(10)

0            [молодёжный, яблоко, оппозиционный, деятельность, становиться, опасный]
1                                                                 [газпром, хватить]
2                                      [бесконечный, партия, четырехмерный, шахматы]
3    [экс-депутат, осудить, фальсификация, выбор, оказаться, член, боевой, братство]
4                 [новый, москва, остаться, территория, экологический, безопасность]
5                         [f1, гран-при, сша, пройти, четыре, машина, стопка, штраф]
6                                [100, ведущий, политик, россия, февраль, 2018, год]
7                                           [закон, культура, принимать, фон, арест]
8                     [насколько, реальный, газовый, подоплёка, сирийский, конфликт]
9                       [фсб, калужский, область, задержать, четверо, участник, иго]
Name: title_norm, dtype: object

In [288]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.11
Recall -  0.21
F1 -  0.14
Jaccard -  0.08


In [292]:
evaluate(data['keywords'],data['title_norm'].apply(lambda x: x[:10]))

Precision -  0.13
Recall -  0.14
F1 -  0.12
Jaccard -  0.07


Качество сильно улучшилось! Можно теперь ещё раз посмотреть, что плохого извлекается.

In [294]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0                   [яблоко, молодёжный, который, год, акция, активист, это, деятельность, наш, политика]
1                               [миллиард, газа, год, куб, метр, газпром, добыча, который, должный, 2020]
2                                          [год, это, книга, роман, писать, тот, выйти, один, мир, очень]
3                      [ким, зинаида, видео, журналист, год, судебный, бывший, свидетель, рубль, который]
4                      [площадь, новый, территория, москва, га, который, тинао, парк, парковый, развитие]
5                                [гонка, который, команда, позиция, место, два, один, круг, из-за, пилот]
6               [место, влияние, рф, позиция, глава, россия, президент, политический, сергей, российский]
7                [культура, закон, стд, который, сфера, предложить, разработать, быть, концепция, услуга]
8                        [газопровод, сирия, год, турция, газа, россия, европа, катар, который, турецкий]
9        [организация, цос, участник, рф, терр

Ещё остались некоторые стоп-слова. Вместо того, чтобы расширять список, давайте попробуем отсеить несуществительные.

In [296]:
def normalize(text):
    
    words = [delete_punct_on_edges(word) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [297]:
data['content_norm'] = data['content'].apply(normalize)

In [375]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(5)]))

Precision -  0.17
Recall -  0.17
F1 -  0.16
Jaccard -  0.1


Ещу улучшения!

In [507]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(10)

0          [яблоко, год, акция, активист, деятельность, политика, власть, человек, жизнь, выборы]
1               [миллиард, год, куб, газа, метр, газпром, добыча, производитель, страна, прогноз]
2                   [год, книга, роман, мир, жанр, стихотворение, перевод, читатель, том, россия]
3         [ким, зинаида, видео, год, журналист, экспертиза, свидетель, рубль, процесс, заседание]
4                   [площадь, москва, территория, га, тинао, парка, парк, столица, развитие, год]
5                  [гонка, команда, позиция, место, круг, пилот, бокс, чемпионат, заезд, дженсон]
6               [место, влияние, рф, позиция, глава, россия, президент, сергей, рейтинг, участие]
7    [культура, закон, сфера, учреждение, изменение, концепция, услуга, проект, сообщество, дело]
8                    [газопровод, сирия, год, турция, газа, европа, россия, катар, поток, проект]
9          [организация, участник, рф, центр, ячейка, эмиссар, область, сирия, деятельность, год]
Name: content_norm, 

Не очень значимые слова все ещё остались. Давайте попробуем отсеять стоп-слова с помощью idf.

In [302]:
data['content_norm_str'] = data['content'].apply(lambda x: ' '.join(normalize(x)))

In [424]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [425]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [426]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [427]:
texts_vectors = tfidf.transform(data['content_norm_str'])

In [428]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-6:-1]]

In [505]:
keywords[:10]

[['яблоко', 'активист', 'акция', 'дарья', 'деятельность'],
 ['миллиард куб', 'куб метр', 'куб', 'газпром', 'газа'],
 ['роман', 'книга', 'жанр', 'стихотворение', 'перевод'],
 ['ким', 'зинаида', 'видео', 'экспертиза', 'свидетель'],
 ['га', 'площадь', 'парка', 'парк', 'площадь га'],
 ['гонка', 'команда', 'бокс', 'баттон', 'позиция'],
 ['го', 'влияние', 'место', 'рф', 'позиция'],
 ['культура', 'закон', 'сфера услуга', 'концепция', 'учреждение'],
 ['газопровод', 'катар', 'сирия', 'турция', 'газа'],
 ['рф организация', 'год участник', 'участник', 'территория сирия', 'ячейка']]

In [430]:
evaluate(data['keywords'], keywords)

Precision -  0.18
Recall -  0.18
F1 -  0.17
Jaccard -  0.1


Сильного улучшения нет. Немного подросла точность. Теперь вместо стоп-слов в ключевые попадают имена и все такое. Иногда это хорощо, а иногда нет (собянин - может быть ключевым словом, а дарья - вряд ли)

## Попробуем графы!

In [431]:
from itertools import combinations

In [445]:
def get_kws(text, top=5, window_size=5):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был направленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        m[i] /= np.sum(m[i])
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    n = np.random.choice(len(vocab))
    for i in range(1000):
        
        j = np.random.randint(1,10)
        if j > 8:
            n = np.random.choice(len(vocab))
        n = take_step(n, m)
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    next_n = np.random.choice(range(rang), p=matrix[n])
    return next_n
    


In [450]:
%%time
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 10, 10))

CPU times: user 50 s, sys: 0 ns, total: 50 s
Wall time: 50 s


In [451]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.11
Recall -  0.22
F1 -  0.14
Jaccard -  0.08


In [506]:
keywords_rw.head(10)

0                           [яблоко, год, активист, акция, сторона, политика, украина, убийство, молодая, жизнь]
1                                 [миллиард, газпром, газа, куб, год, метр, добыча, цена, производитель, страна]
2                                   [год, книга, роман, мениппея, россия, поэзия, москва, работа, перевод, жанр]
3                        [ким, зинаида, видео, журналист, год, подсудимая, свидетель, заседание, губарев, рубль]
4                                 [площадь, москва, га, территория, парк, тинао, регион, год, развитие, столица]
5                            [гонка, команда, позиция, бокс, круг, место, дженсон, ситуация, строчка, чемпионат]
6                            [место, рф, влияние, глава, позиция, президент, россия, сергей, сообщение, алексей]
7    [культура, закон, сфера, изменение, законодательство, человек, концепция, сообщество, проект, председатель]
8                                     [сирия, год, газопровод, россия, турция, европа, катар, ир

In [453]:
import networkx as nx

In [502]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был направленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.degree(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [498]:
matrix, id2word = build_matrix(data['content_norm'][0])

In [503]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 10))

CPU times: user 6.6 s, sys: 0 ns, total: 6.6 s
Wall time: 6.6 s


In [504]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.12
Recall -  0.24
F1 -  0.16
Jaccard -  0.09
